In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

import keras # broken for keras >= 2.0, use 1.2.2
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten, Dense, Activation, Reshape

import utils

Using TensorFlow backend.


In [4]:
import imageio
imageio.plugins.ffmpeg.download()

Imageio: 'ffmpeg.osx' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg.osx (28.8 MB)
Downloading: 8192/30241064 bytes (0.016384/30241064 bytes (0.1%24576/30241064 bytes (0.1%32768/30241064 bytes (0.1%49152/30241064 bytes (0.2%90112/30241064 bytes (0.3%114688/30241064 bytes (0.4147456/30241064 bytes (0.5180224/30241064 bytes (0.6229376/30241064 bytes (0.8311296/30241064 bytes (1.0360448/30241064 bytes (1.2409600/30241064 bytes (1.4458752/30241064 bytes (1.5491520/30241064 bytes (1.6540672/30241064 bytes (1.8573440/30241064 bytes (1.9638976/30241064 bytes (2.1671744/30241064 bytes (2.2720896/30241064 bytes (2.4786432/30241064 bytes (2.6819200/30241064 bytes (2.7884736/30241064 bytes (2.9917504/30241064 bytes (3.0983040/30241064 bytes (3.31032192/30241064 bytes (3.4%1081344/30241064 bytes (3.6%1146880/30241064 bytes (3.8%1196032/30241064 bytes (4.0%1261568/30241064 bytes (4.2%1327104/30241064 byte

In [7]:
#Constructing the yolo model- tiny
keras.backend.set_image_dim_ordering('th')

In [ ]:
#Model Architecture

model = Sequential()
model.add(Convolution2D(16, 3, 3,input_shape=(3,448,448),border_mode='same',subsample=(1,1)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(64,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(128,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(256,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(512,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(1024,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Convolution2D(1024,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Convolution2D(1024,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Flatten())
model.add(Dense(256))
model.add(Dense(4096))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(1470))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 448, 448)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 32, 224, 224)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 112, 112)      18496     
__________

In [ ]:
#loading pre-trained weights

In [13]:
import os
os.chdir("/Users/pratham/Downloads")

In [8]:
import h5py

In [1]:
load_weights(model,'tiny-yolo-voic.weights')

In [ ]:
#Apply the model to a test image
imagePath = './test_images/test1.jpg'
image = plt.imread(imagePath)
image_crop = image[300:650,500:,:]
resized = cv2.resize(image_crop,(448,448))

In [ ]:
batch = np.transpose(resized,(2,0,1))
batch = 2*(batch/255.) - 1
batch = np.expand_dims(batch, axis=0)
out = model.predict(batch)

In [ ]:
boxes = yolo_net_out_to_car_boxes(out[0], threshold = 0.17)


In [ ]:
f,(ax1,ax2) = plt.subplots(1,2,figsize=(16,6))
ax1.imshow(image)
ax2.imshow(draw_box(boxes,plt.imread(imagePath),[[500,1280],[300,650]]))

In [ ]:
#Apply to a video
def frame_func(image):
    crop = image[300:650,500:,:]
    resized = cv2.resize(crop,(448,448))
    batch = np.array([resized[:,:,0],resized[:,:,1],resized[:,:,2]])
    batch = 2*(batch/255.) - 1
    batch = np.expand_dims(batch, axis=0)
    out = model.predict(batch)
    boxes = yolo_net_out_to_car_boxes(out[0], threshold = 0.17)
    return draw_box(boxes,image,[[500,1280],[300,650]])

In [ ]:
project_video_output = './project_video_output.mp4'
clip1 = VideoFileClip("./project_video.mp4")

In [ ]:
lane_clip = clip1.fl_image(frame_func) #NOTE: this function expects color images!!
%time lane_clip.write_videofile(project_video_output, audio=False)